In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'animal-detection-small-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F37898%2F57718%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240401%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240401T071005Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D5dc5840672ab4e41df8be9853b3a5d4fb9d5d88f07ef12c930e1bbb3895c59d937f183dcff2b347573230ae5a19320123a65fdd28771e0b7164a657c7279e0f1b75aa7b5d3e24354d08eb7861ec1dc660599ec1e1273e92b29b628f693c5c98a11daecaea4c70dde1ae10fc45464eb9f722eed330fe437c0e23dc0731f90e4b6f4148e2a6762807eaf75745fd14c8f4d5b33843e45f00b9683249c2a2a3269648fd683a0f15b8b1e4e61f451cb7e947793400ab0bfc2b6d42734621bb3a98ef4ec4e58ca75e3a12629ce1a2925f246322561b6c959c340392803768d75ee625a8974c2c82ffa9fd849d6d01b10a893b32c4c4b35901f3c9ac6001a6a04c80551,animal-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F616920%2F1102439%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240401%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240401T071005Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D688f13f19bee65f54418f8b3baae190cd4eccd0383559360e6bc142d1e814fe7d88755cd3b8e6c451efef862a55cac0b9e182f586899c64a5b6f044196f5355d1b8a4d95731f85bef44f884c0851dd1f939c1a692ada99d569c6a3a6bc8ffdf3dd549421a1d7613161020b1048e2e5d580552ff018b1cf0153e3974e89e7a80be2c2d7319bc5f728e6e17afe0cded6ccb845528b9d991ecb86bce5024cbe70101c0684454fde62d26360596ce944c6620812d1874b21e02b4969ca885f18d0977cd2d5c9e8452506ac5f1f253cb189ca2b4875aa1b11b985eb9e2e18ed15a094c5c85d4f06dbbc2aadc661fd520a03e6432ecce6316770cb4b61db1be14edd37,african-wildlife:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F674157%2F1185810%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240401%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240401T071005Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D42792410a3b125b3ef9dee94baa5fe46fd55f2a2d51fe61887fd7c886799b888e866c2fe55ca81942d41fe47a7893de7f3efca7e41f13fd1b58b0a97cbcf9c22926348728a641120b5d2cafaf03de750670445e0837fa86352489568710e1ef98c7974d87222b2635f1b0e2aa3c0508fa19f290bc0aa726f6dc234fc60bdc9bf2ba59c11c46022a497daec30d3c494448b0b8ec88a998fde5f860c07e5b3a3f50bace82ea0e2dd245c11dffb44078a4760bbb0a1f621cad754219f9a079c2ef127316727ac60174121548ced8b5d5a0cceb6c8d1e80513f431b3a97922c2b08f09c8a34fc10be38b4f807c1cd3a55eeb959cb9375657b9fedd519d42228e35b3,cheetahtigerwolf:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F731756%2F1270998%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240401%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240401T071005Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D63e9fb710940b6ddddd1df6348d414dffad4125c5c8155a0cb327aa1c071d681f08b66b214c5a0cbe9220c3ba04a9ecb7077b2faee6b3e7b70a47c9d8cd65f5a61b7d19dc823b32a06a67c0fbf6e56542a55f31251a6684098c8f7690076120b345756516694e2c0b608f189dc269e502698ac105afdaf104008e37b9b23c34b174fc7a912ec03914caac6ffe8dfa7021f58d81df396bf5a1a0cb27066203524c60492a0ffe199eb07ab08839a942f00d5da5fe584002b2a1b1c102817b66ae0477847e37b1d2213dba023964abb2ae23d9ae61afcfd28ac0c545ce349df12d37a41be6b154aa7aa014ae716c8b1079efbed334f63fa75e6f77b800f469953a9'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 6874991755 bytes downloaded
Downloaded and uncompressed: animal-detection-small-dataset
[==================================================] 434445864 bytes downloaded
Downloaded and uncompressed: animal-classification
[==================================================] 469442673 bytes downloaded
Downloaded and uncompressed: african-wildlife
[==================================================] 2692178008 bytes downloaded
Downloaded and uncompressed: cheetahtigerwolf
Data source import complete.


## Task 1: Importing Libraries

In [ ]:
import keras
import os
import shutil
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from platform import python_version

print('Python version:', python_version())
print('Numpy version:', np.__version__)
print('Seaborn version:', sns.__version__)
from distutils.dir_util import copy_tree
import tensorflow as tf
print('tensorflow version: ',tf.__version__)
print('keras version:', keras.__version__)

Python version: 3.10.12
Numpy version: 1.25.2
Seaborn version: 0.13.1
tensorflow version:  2.15.0
keras version: 2.15.0


In [ ]:
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Dropout
from keras_preprocessing.image import ImageDataGenerator

ModuleNotFoundError: No module named 'keras_preprocessing'

## Task 2: Dataset Creation

In [ ]:
#created data set using console
source='../input/african-wildlife/'
target='./train_data/'
shutil.copytree(source, target)
os.mkdir('test_data')

In [ ]:
# remove unwanted data and create same classed for test_data

path="./train_data/"
for file in os.listdir(path):
    for image in os.listdir(path+file+'/'):
        if '.jpg' not in image:
            os.remove(path+file+'/'+image)
    os.mkdir('./test_data/'+file)


In [ ]:
# create test_data by taking 25% images from data

total_train_images,total_test_images,total_train_classes,total_test_classes=0,0,0,0
path="./train_data/"
for file in os.listdir(path):
    total_train_classes+=1
    total_images=len(os.listdir(path+file+"/"))
    test_image_count=(25/100)*total_images #25% for test and 75% for train
    for i in range(math.ceil(test_image_count)):
        img=random.choice(os.listdir(path+file+'/'))
        shutil.move(path+file+'/'+img,'./test_data/'+file+'/')
        #print(img)
    print(file,total_images,math.ceil(test_image_count))
    total_train_images+=(total_images-math.ceil(test_image_count))
    #print(file,math.ceil(test_image_count))
print("total train images are : ",total_train_images," and total train classes are : ",total_train_classes)

## Task 3: Model Creation

In [ ]:
model = Sequential()

In [ ]:
#inputlayer : apply filters
model.add(Convolution2D(filters=32,
                        kernel_size=(3,3),
                        strides=(1,1),
                        padding='same',
                        activation='relu',
                   input_shape=(32, 32, 1)))

In [ ]:
# pooling layer where we are doing maxpooling
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
#adding one more convolution layer for better model
model.add(Convolution2D(filters=32,
                        kernel_size=(3,3),
                        strides=(1,1),
                        padding='same',
                        activation='relu'
                      ))

In [ ]:
#adding one more Pooling layer for better model
model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
#dropout regularlization
model.add(Dropout(0.5))

In [ ]:
#layer in which we are converting 2d/3d image to 1d image i.e flattening
model.add(Flatten())

In [ ]:
# layer: appling relu to give positive output from here our hidden layerrs starts
model.add(Dense(units=20, activation='relu'))

In [ ]:
#dropout regularlization
model.add(Dropout(0.5))

In [ ]:
# output layer : Since we have to do multi-class classification so we'll apply softmax activation function
# we have 4 classes of animals so output layer would have that many neurons.
model.add(Dense(units=4, activation='softmax'))

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

## Task 4: Image Augmentation

In [ ]:
#url : https://keras.io/api/preprocessing/image/
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)
training_set = train_datagen.flow_from_directory(
        './train_data/',
        target_size=(32,32),
        color_mode="grayscale",
        batch_size=64,
        class_mode='categorical')
test_set = test_datagen.flow_from_directory(
        './test_data/',
        target_size=(32,32),
        color_mode="grayscale",
        batch_size=64,
        class_mode='categorical')

In [ ]:
training_set.class_indices # to see classes of our dataset

## Task 5: Model Training

In [ ]:
history = model.fit(
        training_set,
        steps_per_epoch=(1125/64),
        epochs=100,
        validation_data=test_set,
        validation_steps=(376/64))

## Task 6: Accuracy

In [ ]:
#Graphing our training and validation
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'r', label='Training acc')
plt.plot(epochs, val_accuracy, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
plt.show()

In [ ]:
#model.save("simple_animal_classification_model.h5")#save model

In [ ]:
#from keras.models import load_model
#model=load_model("simple_animal_classification_model.h5")

## Task 7: Testing

In [ ]:
from keras.preprocessing import image
test_image = image.load_img("../input/african-wildlife/zebra/001.jpg",target_size=(32,32),color_mode='grayscale')
test_image


In [ ]:
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image,axis=0)

In [ ]:
result = model.predict(test_image)

my_dict=training_set.class_indices
def get_key(val):
    for key, value in my_dict.items():
         if val == value:
             return key

    return "key doesn't exist"

pred=list(result[0])
for i in range(len(pred)):
    if pred[i]!=0:
        print(get_key(i))